<a href="https://colab.research.google.com/github/GabrielVeloso077/AutoMLAgents/blob/main/LangChain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<table style="margin: auto; background-color: white;">
    <tr>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1S3xpVbkzpBAG51PyuRyIioecvZiHGSap' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1S3xpVbkzpBAG51PyuRyIioecvZiHGSap' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1S3xpVbkzpBAG51PyuRyIioecvZiHGSap' alt="drawing" width="200" />
      </td>
    </tr>
</table>
<font face="Verdana" size=1 color='#707bf8' font-family= 'Alike Angular'> *Este material foi desenvolvido como parte do Projeto de Pesquisa em Parceria com a KUNUMI S/A <font>


<font face="Rage" size=7 color='#707bf8'> LangChain: RAG <font>






###Instalação LangChain

In [ ]:
!pip install --quiet langchain
!pip install --quiet langchain-community

In [ ]:
from langchain_community.chat_models import ChatDeepInfra
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

### Acesso à plataforma DeepInfra

In [ ]:
import os
from getpass import getpass

os.environ["DEEPINFRA_API_TOKEN"] = getpass("Digite sua chave da DeepInfra: ")

Digite sua chave da DeepInfra: ··········


###Geração aumentada por recuperação (RAG)

Uma das aplicações mais poderosas possibilitadas pelos LLMs são os sofisticados chatbots de perguntas e respostas. São aplicativos que podem responder a perguntas sobre informações de fontes específicas. Esses aplicativos utilizam uma técnica conhecida como Geração Aumentada de Recuperação, ou RAG.

Uma aplicação RAG típica tem os seguintes passos:



1. **Carregar:** Primeiro, precisamos carregar nossos dados. Isso é feito com os Document Loaders.
2. **Dividir:** Text splitters dividem grandes documentos em pedaços (chunks) menores. Isso é útil tanto para indexar dados quanto para passá-los para um modelo, pois chunks grandes são mais difíceis de pesquisar e não cabem na janela de contexto finita de um modelo.
3. **Armazenamento:** Precisamos de um lugar para armazenar e indexar nossas divisões, para que possam ser pesquisadas posteriormente. Isso geralmente é feito usando um VectorStore e modelos Embeddings.
4. **Recuperar:** Dada uma entrada do usuário, as divisões relevantes são recuperadas do armazenamento usando um Retriever.
5. **Gerar:** Um ChatModel produz uma resposta usando um prompt que inclui a pergunta com os dados recuperados.



###1. Carregar

Carregamento de um arquivo PDF.

In [ ]:
!pip install pypdf

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader

arquivo = "https://www2.camara.leg.br/legin/fed/lei/1990/lei-8069-13-julho-1990-372211-normaatualizada-pl.pdf"
loader = PyPDFLoader(arquivo)
doc_pdf = loader.load()

print(f'Número de páginas do PDF: {len(doc_pdf)}')

Número de páginas do PDF: 72


Carregamento de um arquivo CSV

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

arquivo = "imdb_movies.csv"  # Faça apload de um arquivo CSV para testar
loader = CSVLoader(arquivo)
doc_csv = loader.load()

print(f'Número de linhas do CSV: {len(doc_csv)}')

Número de linhas do CSV: 1000


Carregamento via URL

In [ ]:
from langchain_community.document_loaders.web_base import WebBaseLoader

url = "https://dcc.ufmg.br/"
loader = WebBaseLoader(url)
doc_url = loader.load()

print(f'Número de páginas: {len(doc_url)}')

Número de páginas: 1


###2. Dividir

Vamos testar algumas formar de dividir um documento.

In [ ]:
texto = """
Modelos de Linguagem de Grande Escala (LLMs, do inglês Large Language Models)
são algoritmos de inteligência artificial treinados com enormes volumes de texto para compreender,
gerar e interagir com linguagem natural. Eles utilizam arquiteturas baseadas em transformers
para captar padrões complexos de uso da linguagem, permitindo aplicações como chatbots,
tradutores automáticos, resumo de textos, geração de código e muito mais.
Seu poder vem da combinação entre o tamanho dos dados usados no treinamento e a escala dos parâmetros do modelo,
o que os torna capazes de realizar tarefas diversas com pouca ou nenhuma adaptação adicional.
"""

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# ✅ CharacterTextSplitter: use quando você quer controle bruto e rápido sobre o tamanho de strings,
# sem se importar com a integridade semântica.

char_split = CharacterTextSplitter(
    chunk_size=50,   # Tamanho máximo (em número de caracteres) de cada chunk gerado.
    chunk_overlap=10, # Número de caracteres que se repetem entre dois chunks consecutivos. Isso é importante para manter o contexto contínuo: o fim de um chunk é repetido no início do próximo.
    separator=""
)

split = char_split.split_text(texto)
split

['Modelos de Linguagem de Grande Escala (LLMs, do i',
 'LLMs, do inglês Large Language Models) \nsão algori',
 'são algoritmos de inteligência artificial treinado',
 'l treinados com enormes volumes de texto para comp',
 'para compreender, \ngerar e interagir com linguage',
 'm linguagem natural. Eles utilizam arquiteturas ba',
 'teturas baseadas em transformers \npara captar padr',
 'aptar padrões complexos de uso da linguagem, permi',
 'gem, permitindo aplicações como chatbots, \ntraduto',
 ', \ntradutores automáticos, resumo de textos, geraç',
 'tos, geração de código e muito mais. \nSeu poder ve',
 'u poder vem da combinação entre o tamanho dos dado',
 'o dos dados usados no treinamento e a escala dos p',
 'cala dos parâmetros do modelo,\no que os torna capa',
 'torna capazes de realizar tarefas diversas com pou',
 'as com pouca ou nenhuma adaptação adicional.']

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ✅ RecursiveCharacterTextSplitter: é o splitter recomendado para documentos com estrutura de linguagem natural,
# pois tenta manter o máximo de contexto relevante por chunk.

char_split = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    separators=["\n\n","\n", " ", ""] # lista de separadores
)

split = char_split.split_text(texto)
split

['Modelos de Linguagem de Grande Escala (LLMs, do',
 '(LLMs, do inglês Large Language Models)',
 'são algoritmos de inteligência artificial',
 'treinados com enormes volumes de texto para',
 'para compreender,',
 'gerar e interagir com linguagem natural. Eles',
 'Eles utilizam arquiteturas baseadas em',
 'em transformers',
 'para captar padrões complexos de uso da',
 'de uso da linguagem, permitindo aplicações como',
 'como chatbots,',
 'tradutores automáticos, resumo de textos, geração',
 'geração de código e muito mais.',
 'Seu poder vem da combinação entre o tamanho dos',
 'dos dados usados no treinamento e a escala dos',
 'dos parâmetros do modelo,',
 'o que os torna capazes de realizar tarefas',
 'tarefas diversas com pouca ou nenhuma adaptação',
 'adaptação adicional.']

In [ ]:
!pip install tiktoken

In [ ]:
from langchain_text_splitters import TokenTextSplitter

# ✅ TokenTextSplitter: use quando você precisa controlar exatamente quantos tokens serão enviados para o modelo,
# por exemplo, para evitar passar do limite de contexto do LLM.

char_split = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10
)

split = char_split.split_text(texto)
split

['\nModelos de Linguagem de Grande Escala (LLMs, do inglês Large Language Models) \nsão algoritmos de inteligência artificial treinados com enormes volumes de texto',
 ' treinados com enormes volumes de texto para compreender, \ngerar e interagir com linguagem natural. Eles utilizam arquiteturas baseadas em transformers \npara captar pad',
 ' em transformers \npara captar padrões complexos de uso da linguagem, permitindo aplicações como chatbots, \ntradutores automáticos,',
 '\ntradutores automáticos, resumo de textos, geração de código e muito mais. \nSeu poder vem da combinação entre o taman',
 ' da combinação entre o tamanho dos dados usados no treinamento e a escala dos parâmetros do modelo,\no que os torna capazes de realizar tarefas',
 ' capazes de realizar tarefas diversas com pouca ou nenhuma adaptação adicional.\n']

Aqui, vamos dividir o documento PDF carregado anteriormente:

In [ ]:
char_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    separators=["\n\n", "\n", ".", " ", ""]
)

split_pdf = char_split.split_documents(doc_pdf)

###3. Armazenar

Primeiro vamos trabalhar com os embeddings.

Embeddings criam uma representação vetorial de um trecho de texto. Isso é útil porque nos permite pensar no texto no espaço vetorial e realizar ações como busca semântica, na qual procuramos trechos de texto mais semelhantes no espaço vetorial.

A classe Embeddings é uma classe projetada para interagir com embedding models.

#### 💸 Modelos de Embeddings da DeepInfra (ordenados por preço)

| Modelo                                                                 | Preço (Mtoken) |
|------------------------------------------------------------------------|------------------|
| [BAAI/bge-base-en-v1.5](https://deepinfra.com/models/BAAI/bge-base-en-v1.5)                         | 0.005           |
| [intfloat/e5-base-v2](https://deepinfra.com/models/intfloat/e5-base-v2)                             | 0.005           |
| [thenlper/gte-base](https://deepinfra.com/models/thenlper/gte-base)                                 | 0.005           |
| [sentence-transformers/all-MiniLM-L6-v2](https://deepinfra.com/models/sentence-transformers/all-MiniLM-L6-v2) | 0.005           |
| [sentence-transformers/all-MiniLM-L12-v2](https://deepinfra.com/models/sentence-transformers/all-MiniLM-L12-v2) | 0.005           |
| [sentence-transformers/all-mpnet-base-v2](https://deepinfra.com/models/sentence-transformers/all-mpnet-base-v2) | 0.005           |
| [sentence-transformers/clip-ViT-B-32](https://deepinfra.com/models/sentence-transformers/clip-ViT-B-32) | 0.005           |
| [sentence-transformers/clip-ViT-B-32-multilingual-v1](https://deepinfra.com/models/sentence-transformers/clip-ViT-B-32-multilingual-v1) | 0.005           |
| [sentence-transformers/paraphrase-MiniLM-L6-v2](https://deepinfra.com/models/sentence-transformers/paraphrase-MiniLM-L6-v2) | 0.005           |
| [shibing624/text2vec-base-chinese](https://deepinfra.com/models/shibing624/text2vec-base-chinese) | 0.005           |
| [sentence-transformers/multi-qa-mpnet-base-dot-v1](https://deepinfra.com/models/sentence-transformers/multi-qa-mpnet-base-dot-v1) | 0.005           |
| [BAAI/bge-large-en-v1.5](https://deepinfra.com/models/BAAI/bge-large-en-v1.5)                       | 0.010           |
| [intfloat/e5-large-v2](https://deepinfra.com/models/intfloat/e5-large-v2)                           | 0.010           |
| [thenlper/gte-large](https://deepinfra.com/models/thenlper/gte-large)                               | 0.010           |
| [BAAI/bge-en-icl](https://deepinfra.com/models/BAAI/bge-en-icl)                                     | 0.010           |
| [BAAI/bge-m3](https://deepinfra.com/models/BAAI/bge-m3)                                              | 0.100           |


In [ ]:
from langchain_community.embeddings import DeepInfraEmbeddings

emb_model = DeepInfraEmbeddings(model_id="BAAI/bge-base-en-v1.5")

Agora vamos utilizar o banco de dados Chroma (vector store) para armazenar nossos embeddings e realizar pesquisas de similaridade.

In [ ]:
!pip install --upgrade --quiet  sqlite-vec

In [ ]:
# Criando uma lista dos documentos que foram divididos para inserir no banco de dados
textos = []
metadados = []

for item in split_pdf:
    textos.append(item.page_content)
    metadados.append(item.metadata)

In [ ]:
from langchain_community.vectorstores import SQLiteVec

vector_store = SQLiteVec.from_texts(
    texts=textos,
    metadatas=metadados,
    embedding=emb_model,
    table="tabela_vetorial",
    db_file="sqlite-vec.db",
)

In [ ]:
# Abrindo o banco de dados
vector_store = SQLiteVec(
    connection=None,
    embedding=emb_model,
    table="tabela_vetorial",
    db_file="sqlite-vec.db",
)

In [ ]:
pergunta = "criança"

docs = vector_store.similarity_search(pergunta, k=10)   # Número de documentos similares que deve ser retornado
print(f"Número de documentos retornados: {len(docs)}")

Número de documentos retornados: 10


In [ ]:
for doc in docs:   # Visualização dos documentos retornados
    print(doc.page_content)
    print(f"==== {doc.metadata}\n")

gestantes, de forma transvers al, integral e intersetorial com as demais linhas de cuidado 
direcionadas à mulher e à criança. (Parágrafo acrescido pela Lei nº 13.257, de 8/3/2016) 
§ 3 º A atenção odontológica à criança terá função educativa protetiva e será prestada, 
inicialmente, antes de o bebê nascer, por meio de aconselhamento pré -natal, e, posteriormente, no
==== {'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2017-04-25T15:19:20-03:00', 'title': 'LEI Nº 8', 'author': 'p_6140', 'moddate': '2017-04-25T15:19:20-03:00', 'source': 'https://www2.camara.leg.br/legin/fed/lei/1990/lei-8069-13-julho-1990-372211-normaatualizada-pl.pdf', 'total_pages': 72, 'page': 3, 'page_label': '4'}

gestantes, de forma transvers al, integral e intersetorial com as demais linhas de cuidado 
direcionadas à mulher e à criança. (Parágrafo acrescido pela Lei nº 13.257, de 8/3/2016) 
§ 3 º A atenção odontológica à criança terá função educativa protetiva e será prest

###4. Recuperação

In [ ]:
from langchain.prompts import ChatPromptTemplate

retriever = vector_store.as_retriever()

pergunta = "Quais pessoas podem ser consideradas crianças?"

chat_template = ChatPromptTemplate.from_messages([
    ("system", """Use os contextos abaixo para responder a questão no fim.
                Se não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta.
                Use no máximo três sentenças para manter a resposta mais simples o possível.
                Sempre diga "obrigado por perguntar" no fim da resposta.

                {context}

                """),
    ("human", "{pergunta}")
  ]
)

# BUSCA POR DOCUMENTOS SIMILARES

docs = retriever.invoke(pergunta)

docs_text = " ".join(d.page_content for d in docs)  # COMBINA OS DOCUMENTOS RETORNADOS EM UMA STRING ÚNICA

chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

chat.invoke(chat_template.format_messages(context=docs_text, pergunta=pergunta)).content

'As pessoas até 12 anos podem ser consideradas crianças.\n\nobrigado por perguntar.'